In [2]:
import spacy
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\spradha5\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\spradha5\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
#doc = nlp('This is some text that I')
#doc.vector.tolist()

In [5]:
#training dataset
df = pd.read_csv('P1_training_set.csv')
e1 = df['Event 1']
e2 = df['Event 2']
label = df['Label']

In [6]:
def gen_vec(s):
    doc = nlp(s)
    vec = doc.vector
    return vec

In [7]:
#training vectors
xtrain = []
ytrain = []

for i in range(e1.size):
    s1 = e1.loc[i]
    s2 = e2.loc[i]
    lab = label.loc[i]
    vec1 = gen_vec(s1)
    vec2 = gen_vec(s2)
    vec = vec1.tolist()+vec2.tolist()
    #print(len(vec))
    
    xtrain.append(vec)
    ytrain.append(lab)
    
xtrain = np.array(xtrain).astype(np.int32)
ytrain = np.array(ytrain).astype(np.int32)

In [8]:
#Random Forrest Training
rfc = RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features='auto')
rfc.fit(xtrain, ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [13]:
#SVM Training
#stf = LinearSVC(random_state=0, tol=1e-5,dual=True,max_iter=50000,multi_class='ovr')
#stf.fit(xtrain,ytrain)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=50000,
          multi_class='ovr', penalty='l2', random_state=0, tol=1e-05,
          verbose=0)

In [9]:
#testing dataset
df2 = pd.read_csv('P1_testing_set.csv')
e1t = df2['Event 1']
e2t = df2['Event 2']
labelt = df2['Label']

In [10]:
#testing
xtest = []
ytest = []

for i in range(e1t.size):
    s1 = e1t.loc[i]
    s2 = e2t.loc[i]
    lab = labelt.loc[i]
    vec1 = gen_vec(s1)
    vec2 = gen_vec(s2)
    vec = vec1.tolist()+vec2.tolist()
    
    xtest.append(vec)
    ytest.append(lab)
    
xtest = np.array(xtest).astype(np.int32)
ytest = np.array(ytest).astype(np.int32)

In [11]:
#Random Forrest Testing
ypred = rfc.predict(xtest)
print(rfc.score(xtest,ytest)*100)

46.666666666666664


In [13]:
#SVM Testing
#ypred = stf.predict(xtest)
#print(stf.score(xtest,ytest)*100)

In [14]:
#Performance Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('\n Confusion Matrix: \n', confusion_matrix(ytest,ypred,labels=[0,1,2]))
print('\n Accuracy: ', accuracy_score(ytest,ypred)*100)
print('\n Precision, Recall, F1-score: \n',classification_report(ytest,ypred))


 Confusion Matrix: 
 [[  0  98  89]
 [  0 242 139]
 [  0 154 178]]

 Accuracy:  46.666666666666664


C:\Users\spradha5\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



 Precision, Recall, F1-score: 
               precision    recall  f1-score   support

           0       0.00      0.00      0.00       187
           1       0.49      0.64      0.55       381
           2       0.44      0.54      0.48       332

    accuracy                           0.47       900
   macro avg       0.31      0.39      0.35       900
weighted avg       0.37      0.47      0.41       900

